# Import Data

In [107]:
import pandas as pd
import numpy as np
import nltk as nk
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [108]:
from itertools import chain

In [109]:
#from google.colab import drive
#drive.mount('/content/drive')

In [110]:
df = pd.read_excel("Data-FakeRealCOVID.xlsx")
#df = pd.read_excel("/content/drive/MyDrive/PROJET_IA/Data-FakeRealCOVID.xlsx")

In [111]:
df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


# Data cleaning

**Harmonization**


In [6]:
def lowerTweet(tweet):
    return tweet.lower()

In [7]:
df.tweet.loc[0]

'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'

In [8]:
lowerTweet(df.tweet.loc[0])

'the cdc currently reports 99031 deaths. in general the discrepancies in death counts between different sources are small and explicable. the death toll stands at roughly 100000 people today.'

**Remove URLS**

In [9]:
# In order to remove any URL within a string in Python, you can use this RegEx function :
def removeURLs(tweet):
        return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet)


In [10]:
df.loc[4,'tweet']

'Populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC. https://t.co/1pYW6cWRaS'

In [11]:
removeURLs(df.loc[4,'tweet'])

'Populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC.  '

**Remove emojis**

In [12]:
#i took an example to work on 
example = df.loc[89,'tweet']
example

'📢#CoronaVirusUpdates: 📍#COVID19 India Tracker (As on 7 September 2020 08:00 AM) ➡️Confirmed cases: 4204613 ➡️Recovered: 3250429 (77.3%)👍 ➡️Active cases: 882542 (21.0%) ➡️Deaths: 71642 (1.7%) #IndiaFightsCorona #IndiaWillWin #StaySafe Via @MoHFW_INDIA https://t.co/WVQslxqBPC'

In [13]:
# Remove all traces of emoji from a text file.

'''def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
remove_emoji(df.loc[6416,'tweet'])'''

'def remove_emoji(string):\n    emoji_pattern = re.compile("["\n                               u"😀-🙏"  # emoticons\n                               u"🌀-🗿"  # symbols & pictographs\n                               u"🚀-\U0001f6ff"  # transport & map symbols\n                               u"\U0001f1e0-🇿"  # flags (iOS)\n                               u"─-⯯"  # chinese char\n                               u"✂-➰"\n                               u"✂-➰"\n                               u"Ⓜ-🉑"\n                               u"🤦-🤷"\n                               u"𐀀-\U0010ffff"\n                               u"♀-♂"\n                               u"☀-⭕"\n                               u"\u200d"\n                               u"⏏"\n                               u"⏩"\n                               u"⌚"\n                               u"️"  # dingbats\n                               u"〰"\n                               "]+", flags=re.UNICODE)\n    return emoji_pattern.sub(r\'\', string)\nremov

In [14]:
#the easiet solution we found is to turn text incode into ascii wich does not include emojis and ignore all other characters that are not defined on ascii encode
#we use encode function which turns string into ascii code and ignoring none ascii characters
#then decode to make the ascii code a string again
def toAscii(tweet):
  return  (tweet.encode('ascii',errors='ignore')).decode('ascii') 


In [15]:
toAscii(example)

'#CoronaVirusUpdates: #COVID19 India Tracker (As on 7 September 2020 08:00 AM) Confirmed cases: 4204613 Recovered: 3250429 (77.3%) Active cases: 882542 (21.0%) Deaths: 71642 (1.7%) #IndiaFightsCorona #IndiaWillWin #StaySafe Via @MoHFW_INDIA https://t.co/WVQslxqBPC'

**Remove punctuation & special characters**

In [16]:
#removing all numbers ,ponctuation & special characters such as @ and #
#refrence to https://www.kerryr.net/pioneers/ascii3.htm

def removeSpecialChar(text):
    text = list(text)
    
    #remove special character & ponctuations & numbers
    listOfSpecialChars =  [*range(33,48),*range(58,65),*range(91,97),*range(123,127)]
    listOfSpecialChars.remove(39) #dont include ' in the list
    
    text = [char for char in text if (ord(char) not in listOfSpecialChars and not char.isdigit())] 
    text = ''.join(text)
    
    return text

In [17]:
#lets test on this example
df.loc[264].tweet

"More detail on this from one of @COVID19Tracking's sharpest observers @jordan_zino from @GavinNewsom's livestream. https://t.co/l2s3AloB28 https://t.co/oZuw8XckHd"

In [18]:
removeSpecialChar(df.loc[264,'tweet'])

"More detail on this from one of COVIDTracking's sharpest observers jordanzino from GavinNewsom's livestream httpstcolsAloB httpstcooZuwXckHd"

**remove apostrophe**

In [19]:
def removeApos(text):
    text = text.replace("'",'')
    return text

In [20]:
#lets test on this example
df.loc[264].tweet

"More detail on this from one of @COVID19Tracking's sharpest observers @jordan_zino from @GavinNewsom's livestream. https://t.co/l2s3AloB28 https://t.co/oZuw8XckHd"

In [21]:
removeApos(df.loc[264,'tweet'])

'More detail on this from one of @COVID19Trackings sharpest observers @jordan_zino from @GavinNewsoms livestream. https://t.co/l2s3AloB28 https://t.co/oZuw8XckHd'

In [22]:
# Thanks to https://www.kaggle.com/rftexas/text-only-kfold-bert
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard","ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard", "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [23]:
# Thanks to https://www.kaggle.com/rftexas/text-only-kfold-bert
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

In [24]:
def convert_abbrev_in_text(text):
    tokens = text.split(' ')
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

# Dimensionality Reduction




**Remove one letter words**

In [25]:
def removeLetter(text):
    text = text.split()
    text = [word for word in text if  len(word) > 1]
    text = ' '.join(text)
    return text

**Remove stopWords**

In [26]:
#nk.download('stopwords')

from nltk.corpus import stopwords

In [27]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [28]:
def removeStopWords(text):
    sw = stopwords.words("english")
    text = text.split()
    text = [word for word in text if word not in sw]
    return ' '.join(text)



In [29]:
df.loc[51].tweet

'132 new cases of #COVID19Nigeria; Lagos-52 Gombe-27 Plateau-17 Kwara-10 Enugu-9 Ogun-9 Katsina-3 Ekiti-2 Bauchi-1 Osun-1 Rivers-1 56388 confirmed 44337 discharged 1083 deaths https://t.co/Enpey5u7xh'

In [30]:
removeStopWords(df.loc[51].tweet)

'132 new cases #COVID19Nigeria; Lagos-52 Gombe-27 Plateau-17 Kwara-10 Enugu-9 Ogun-9 Katsina-3 Ekiti-2 Bauchi-1 Osun-1 Rivers-1 56388 confirmed 44337 discharged 1083 deaths https://t.co/Enpey5u7xh'

**Stemming**

In [31]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [32]:
def stem(text):
    text = text.split()
    text= [ps.stem(word) for word in text]
    return ' '.join(text)

In [33]:
removeSpecialChar(df.loc[0,'tweet']) 

'The CDC currently reports  deaths In general the discrepancies in death counts between different sources are small and explicable The death toll stands at roughly  people today'

In [34]:
stem(removeSpecialChar(df.loc[0,'tweet']) )

'the cdc current report death in gener the discrep in death count between differ sourc are small and explic the death toll stand at roughli peopl today'

# Data Preprocessing

In [35]:
def preprocess(text):
    text = lowerTweet(text)
    text = removeURLs(text)
    text = toAscii(text)
    text = removeSpecialChar(text)
    text = removeStopWords(text)
    text = removeApos(text)
    text = removeStopWords(text)
    text = removeLetter(text)
    text = stem(text)
    text = convert_abbrev_in_text(text)
    return text

In [36]:
df.loc[209,'tweet']

'Children under 18 years represent about 8.5% of reported #COVID19 cases with few deaths compared to other age groups &amp; usually mild disease. 👉 https://t.co/Cs7uxQRRMN https://t.co/l4gqfttYmg'

In [37]:
preprocess(df.loc[209,'tweet'])

'children year repres report covid case death compar age group amp usual mild diseas'

# Tokenization (Not Necessary)

**Tokenize**

In [38]:
nk.download('punkt')

[nltk_data] Downloading package punkt to /home/meedbek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
def tokenize(text):
    return nk.word_tokenize(text)

In [40]:
df.loc[51,'tweet']

'132 new cases of #COVID19Nigeria; Lagos-52 Gombe-27 Plateau-17 Kwara-10 Enugu-9 Ogun-9 Katsina-3 Ekiti-2 Bauchi-1 Osun-1 Rivers-1 56388 confirmed 44337 discharged 1083 deaths https://t.co/Enpey5u7xh'

In [41]:
tokenize(preprocess(df.loc[51,'tweet']))

['new',
 'case',
 'covidnigeria',
 'lago',
 'gomb',
 'plateau',
 'kwara',
 'enugu',
 'ogun',
 'katsina',
 'ekiti',
 'bauchi',
 'osun',
 'river',
 'confirm',
 'discharg',
 'death']

In [42]:
tokenize('i ll see what would this give me')

['i', 'll', 'see', 'what', 'would', 'this', 'give', 'me']

**Bag of Words**

In [43]:
def bagOfWords(df):
    tokens = []
    bag = {}
    for tweet in df.tweet:
        tokens += tokenize(preprocess(tweet))
    for token in tokens:
        if token not in bag.keys():
            bag[token] = 1
        else:
            bag[token] += 1
    return bag

In [44]:
bagOfWords(df)

{'cdc': 174,
 'current': 151,
 'report': 1041,
 'death': 814,
 'gener': 44,
 'discrep': 3,
 'count': 47,
 'differ': 79,
 'sourc': 47,
 'small': 29,
 'explic': 1,
 'toll': 33,
 'stand': 56,
 'roughli': 10,
 'peopl': 755,
 'today': 415,
 'state': 958,
 'rise': 88,
 'last': 229,
 'tuesday': 16,
 'southern': 6,
 'polit': 12,
 'correct': 23,
 'woman': 51,
 'almost': 40,
 'use': 278,
 'pandem': 319,
 'excus': 6,
 'reus': 3,
 'plastic': 6,
 'bag': 12,
 'coronaviru': 1651,
 'nashvil': 16,
 'indiafightscorona': 320,
 'covid': 3857,
 'test': 1680,
 'laboratori': 105,
 'india': 517,
 'th': 122,
 'august': 73,
 'done': 45,
 'profbhargava': 7,
 'dg': 11,
 'icmrdelhi': 105,
 'staysaf': 137,
 'indiawillwin': 116,
 'popul': 77,
 'larg': 49,
 'case': 2024,
 'look': 80,
 'new': 1105,
 'per': 138,
 'million': 210,
 'smaller': 7,
 'show': 310,
 'california': 65,
 'texa': 52,
 'al': 2,
 'ar': 2,
 'id': 4,
 'ks': 3,
 'ky': 3,
 'la': 17,
 'miss': 28,
 'nv': 5,
 'sc': 3,
 'act': 51,
 'found': 89,
 'averag': 1

#  Weighting based on TF-IDF ( TfidfVectorizer ) 



In [45]:
#https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
corpus = []
for  tweet in df.tweet :
  text = preprocess(tweet)
  corpus.append(text)



In [46]:
corpus


['cdc current report death gener discrep death count differ sourc small explic death toll stand roughli peopl today',
 'state report death small rise last tuesday southern state report death',
 'polit correct woman almost use pandem excus reus plastic bag coronaviru nashvil',
 'indiafightscorona covid test laboratori india th august test done profbhargava dg icmrdelhi staysaf indiawillwin',
 'popul state gener larg case count look new case per million today smaller state show case per million california texa al ar id ks ky la miss nv sc',
 'covid act found averag person illinoi covid infect peopl data show infect growth rate declin time factor stayathom order restrict put place',
 'test posit covid symptom stay home away peopl learn cdc recommend around other covid infect',
 'obama call trump coronaviru respons chaotic disast',
 'clearli obama administr leav kind game plan someth like',
 'retractionhydroxychloroquin chloroquin without macrolid treatment covid multin registri analysi la


**Creating TF-IDF(Term Frequency and Inverse Document Frequency)**



In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_df=2,min_df=2).fit(corpus)


**Extracting all tokens i.e Features**

In [48]:
count_tokens=tfidf.get_feature_names_out()
count_tokens

array(['aaioffici', 'aaj', 'aajtak', ..., 'zombi', 'zoo', 'zookeep'],
      dtype=object)


**Transforming the corpus into Vector**

In [49]:
article_vect = tfidf.transform(corpus)
article_vect.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [50]:
tfidf.idf_

array([8.66871686, 8.66871686, 8.66871686, ..., 8.66871686, 8.66871686,
       8.66871686])

In [51]:
len(tfidf.idf_)

1243

**Displaying the final Converted Vector with Features**

In [52]:
df_tfidf_vect=pd.DataFrame(data=article_vect.toarray(),columns=count_tokens)
df_tfidf_vect

,aaioffici,aaj,aajtak,aamaadmiparti,ababa,ableg,abpnew,absent,ac,accid,...,yeardrtedro,yell,yield,yorker,youll,zee,zithromax,zombi,zoo,zookeep
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
def TfIdfVec(df):
    corpus = []
    for  tweet in df.tweet :
      text = preprocess(tweet)
      corpus.append(text)

    tfidf = TfidfVectorizer().fit(corpus)

    tfidf_tokens = tfidf.get_feature_names_out()
    article_vect = tfidf.transform(corpus)

    df_tfidf_vect = pd.DataFrame(data=article_vect.toarray(),columns=tfidf_tokens)

    return df_tfidf_vect,tfidf

In [54]:
#calculer tf-idf manuellement (pour pouvoir le calculer pour d'autre data)
corpus[0]

'cdc current report death gener discrep death count differ sourc small explic death toll stand roughli peopl today'

In [55]:
#on calcule le tf de cdc 
tokens = corpus[0].split(' ')
tf = len([token for token in tokens if token=='cdc' ])/len(tokens)
tf

0.05555555555555555

In [56]:
#index de cdc dans les la liste des tokens
index = np.where(count_tokens == 'cdc')[0]
index

array([], dtype=int64)

In [57]:
#la valeur de son idf
cdc_idf = tfidf.idf_[index]
cdc_idf

array([], dtype=float64)

In [58]:
tfidf.transform(['aa	gcg aadajoli gvg	aavioffici	 vcgvg aaj	aajtak']).toarray().shape

(1, 1243)

# Weighting based on TF ( CountVectorizer )

In [59]:
#https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a
corpus = []
for  tweet in df.tweet :
  text = preprocess(tweet)
  corpus.append(text)



In [60]:
corpus


['cdc current report death gener discrep death count differ sourc small explic death toll stand roughli peopl today',
 'state report death small rise last tuesday southern state report death',
 'polit correct woman almost use pandem excus reus plastic bag coronaviru nashvil',
 'indiafightscorona covid test laboratori india th august test done profbhargava dg icmrdelhi staysaf indiawillwin',
 'popul state gener larg case count look new case per million today smaller state show case per million california texa al ar id ks ky la miss nv sc',
 'covid act found averag person illinoi covid infect peopl data show infect growth rate declin time factor stayathom order restrict put place',
 'test posit covid symptom stay home away peopl learn cdc recommend around other covid infect',
 'obama call trump coronaviru respons chaotic disast',
 'clearli obama administr leav kind game plan someth like',
 'retractionhydroxychloroquin chloroquin without macrolid treatment covid multin registri analysi la


**Creating Count_Vectorizer**



In [61]:
from sklearn.feature_extraction.text import CountVectorizer

count=CountVectorizer().fit(corpus)


**Extracting all tokens i.e Features**

In [62]:
count_tokens=count.get_feature_names_out()
count_tokens

array(['aa', 'aadajoli', 'aaioffici', ..., 'zubymus', 'zydu', 'zyphr'],
      dtype=object)


**Transforming the corpus into Vector**

In [63]:
article_vect = count.transform(corpus)
article_vect.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

**Displaying the final Converted Vector with Features**

In [64]:

df_count_vect=pd.DataFrame(data=article_vect.toarray(),columns=count_tokens)


df_count_vect

,aa,aadajoli,aaioffici,aaj,aajtak,aamaadmiparti,aamctoday,aamir,aamp,aaradhya,...,zone,zonecityspecif,zoo,zoodirector,zookeep,zoolog,zoom,zubymus,zydu,zyphr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6418,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
def CountVec(df):
    corpus = []
    for  tweet in df.tweet :
      text = preprocess(tweet)
      corpus.append(text)

    count = CountVectorizer().fit(corpus)

    count_tokens = count.get_feature_names_out()
    article_vect = count.transform(corpus)

    df_count_vect = pd.DataFrame(data=article_vect.toarray(),columns=count_tokens)

    return df_count_vect

# Dimensionality Reduction based on Frequency Count


The goal is to reduce the dimension of the vectorized data by removing columns (features) based on their document frequency in other words if   the values of a specific column are not different from 0 in more than a specific threshold(seuil) we remove the column from the DateFrame 

**Vectorizing the data**

In [66]:
dfVec = TfIdfVec(df)

In [67]:
dfVec

,aa,aadajoli,aaioffici,aaj,aajtak,aamaadmiparti,aamctoday,aamir,aamp,aaradhya,...,zone,zonecityspecif,zoo,zoodirector,zookeep,zoolog,zoom,zubymus,zydu,zyphr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.418815,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Selecting the list of columns that do not achieve a specific threshold(seuil)**

In [68]:
subDf = dfVec[dfVec != 0]
subDf

,aa,aadajoli,aaioffici,aaj,aajtak,aamaadmiparti,aamctoday,aamir,aamp,aaradhya,...,zone,zonecityspecif,zoo,zoodirector,zookeep,zoolog,zoom,zubymus,zydu,zyphr
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.418815,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:

subDf.count(axis = 0)

aa           1
aadajoli     1
aaioffici    2
aaj          2
aajtak       2
            ..
zoolog       1
zoom         5
zubymus      1
zydu         1
zyphr        1
Length: 9638, dtype: int64

In [70]:

map = subDf.count(axis = 0) <=2  
map2 = subDf.count(axis = 0) >= 9000
map + map2

aa            True
aadajoli      True
aaioffici     True
aaj           True
aajtak        True
             ...  
zoolog        True
zoom         False
zubymus       True
zydu          True
zyphr         True
Length: 9638, dtype: bool

**Droping the columns selected**

In [71]:
dfVec.drop(dfVec.columns[map],axis=1)

,aamir,aamp,abandon,abbott,abia,abil,abl,abort,absenc,absolut,...,youv,yr,zack,zamfara,zealand,zero,zika,zinc,zone,zoom
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6416,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6417,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
6418,0.418815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [72]:
columns_dropped=dfVec.columns[map+map2]
columns_dropped

Index(['aa', 'aadajoli', 'aaioffici', 'aaj', 'aajtak', 'aamaadmiparti',
       'aamctoday', 'aaradhya', 'ababa', 'abacha',
       ...
       'zomato', 'zombi', 'zonecityspecif', 'zoo', 'zoodirector', 'zookeep',
       'zoolog', 'zubymus', 'zydu', 'zyphr'],
      dtype='object', length=6215)

In [102]:
def redDim(dfVec,seuil1,seuil2):
    subDf = dfVec[dfVec != 0]
    map = subDf.count(axis = 0) <= seuil1 
    map2 = subDf.count(axis = 0) >= (dfVec.shape[0] - seuil2) 
    resultDf = dfVec.drop(dfVec.columns[map+map2],axis=1)
    columns_dropped=dfVec.columns[map+map2]
    
    return resultDf,columns_dropped

In [113]:
newDF,tfIdfObject = TfIdfVec(df)

In [114]:
newDF.shape[1]

9638

In [103]:
newDF,columns = redDim(newDF,5,5)

In [115]:
import pickle

In [116]:
pickle.dump(tfIdfObject, open("tfidf.pkl", "wb"))
pickle.dump(columns, open("drop_columns.pkl", "wb"))

In [117]:
#newDF.to_csv('/content/drive/MyDrive/PROJET_IA/newDF.csv')
newDF.to_csv('newDF.csv')